In [ ]:
# === Reproducibility Setup =====================================================
# Install dependencies (Colab-friendly) and set up folders/config.
import subprocess, os, json, pathlib, yaml, random
from pathlib import Path

IN_COLAB = False
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    pass

if IN_COLAB:
    _ = subprocess.run(
        "pip -q install -r https://raw.githubusercontent.com/huanyuhuanyuhuanyu-arch/UA_Geotransformer_for_ES/main/requirements.txt",
        shell=True, check=False
    )

ROOT = Path.cwd()
(DATA_DIR := ROOT/"data").mkdir(exist_ok=True)
for p in ["raw", "processed"]:
    (DATA_DIR/p).mkdir(exist_ok=True, parents=True)
(OUT_DIR := ROOT/"outputs").mkdir(exist_ok=True)

CFG_PATH = ROOT/"configs"/"default.yaml"
with open(CFG_PATH, "r", encoding="utf-8") as f:
    CFG = yaml.safe_load(f)

SEED = int(CFG.get("params", {}).get("random_seed", 42))
def set_seed(seed=42):
    import numpy as np
    random.seed(seed); np.random.seed(seed)
    try:
        import torch
        torch.manual_seed(seed)
        if hasattr(torch, "cuda") and torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
    except Exception:
        pass
set_seed(SEED)
print("[setup] paths:", json.dumps(CFG.get("paths", {}), indent=2))
print("[setup] seed:", SEED)
# ==============================================================================


In [ ]:
# === Data Download (no checksum) ==============================================
# Downloads required data files from Zenodo into data/raw if they don't exist.
# Safe to re-run. No integrity check (by design).

import pathlib, urllib.request

RAW = pathlib.Path("data/raw")
RAW.mkdir(parents=True, exist_ok=True)

FILES = {
    "2010_9.tif": "https://zenodo.org/records/17251210/files/2010_9.tif?download=1",
    "2017_9.tif": "https://zenodo.org/records/17251210/files/2017_9.tif?download=1",
    "aspect.tif": "https://zenodo.org/records/17251210/files/aspect.tif?download=1",
    "distance_rail.tif": "https://zenodo.org/records/17251210/files/distance_rail.tif?download=1",
    "distance_road.tif": "https://zenodo.org/records/17251210/files/distance_road.tif?download=1",
    "distance_water.tif": "https://zenodo.org/records/17251210/files/distance_water.tif?download=1",
    "ecoreserve.tif": "https://zenodo.org/records/17251210/files/ecoreserve.tif?download=1",
    "elevation.tif": "https://zenodo.org/records/17251210/files/elevation.tif?download=1",
    "pop_density.tif": "https://zenodo.org/records/17251210/files/pop_density.tif?download=1",
    "runoff.tif": "https://zenodo.org/records/17251210/files/runoff.tif?download=1",
    "slope.tif": "https://zenodo.org/records/17251210/files/slope.tif?download=1",
    "wetland.tif": "https://zenodo.org/records/17251210/files/wetland.tif?download=1",
}

def maybe_download(name: str, url: str, outdir: pathlib.Path) -> pathlib.Path:
    dst = outdir / name
    if dst.exists():
        print(f"[skip] {name} already exists.")
        return dst
    print(f"[downloading] {name} → {dst}")
    urllib.request.urlretrieve(url, dst)
    return dst

for fname, url in FILES.items():
    maybe_download(fname, url, RAW)

print("[data] Ready.")
# ==============================================================================


# Your analysis starts here
Load data from `data/raw`, follow your pipeline, and export figures/tables to `outputs/`.
